In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import cv2
import numpy as np
from os import listdir as ld
from os.path import join as pj
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data

# Dataset
from dataset.detection.dataset import insects_dataset_from_voc_style_txt, collate_fn
# Loader
from IO.loader import load_path, load_images, load_annotations_path, load_annotations, get_anno_recs
# Predict
from model.refinedet.utils.predict import test_prediction_with_cls
# utils
from IO.utils import refine_result_by_ovthresh_with_cls, result_formatter_with_cls
# Evaluate
from evaluation.det2cls.evaluate import create_imwise_gt, get_cls_accuracy_per_class_with_cls
from evaluation.det2cls.visualize import vis_detections

# Test Config

In [3]:
class args:
    # experiment name
    experiment_name = "crop_b2_2_4_8_16_32_im512_cls"
    # paths
    data_root = "/home/tanida/workspace/Insect_Phenology_Detector/data"
    test_image_root = "/home/tanida/workspace/Insect_Phenology_Detector/data/test_refined_images"
    model_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/output_model/detection/RefineDet", experiment_name)
    figure_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/figure/det2cls/RefineDet_only", "crop_b2_2_4_8_16_32_im512_cls")
    test_anno_folders = ["annotations_4"]
    # training config
    input_size = 512 # choices=[320, 512, 1024]
    crop_num = (5, 5)
    tcb_layer_num = 5
    rm_last = True

# Model Config

In [4]:
if args.tcb_layer_num == 4 and args.rm_last == False:
    from model.refinedet.config import tcb_4_rm_false as insect_refinedet
elif args.tcb_layer_num == 4 and args.rm_last == True:
    from model.refinedet.config import tcb_4_rm_true as insect_refinedet
elif args.tcb_layer_num == 5 and args.rm_last == False:
    from model.refinedet.config import tcb_5_rm_false as insect_refinedet
elif args.tcb_layer_num == 5 and args.rm_last == True:
    from model.refinedet.config import tcb_5_rm_true as insect_refinedet
elif args.tcb_layer_num == 6 and args.rm_last == False:
    from model.refinedet.config import tcb_6_rm_false as insect_refinedet

# Set cuda

In [5]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

# Model

In [6]:
if args.rm_last == True:
    from model.refinedet.refinedet_rmlast import build_refinedet
else:
    from model.refinedet.refinedet import build_refinedet

### Make data

In [7]:
print('Loading dataset for test ...')
test_dataset = insects_dataset_from_voc_style_txt(args.test_image_root, args.input_size, args.crop_num, "RefineDet", training=False)
test_data_loader = data.DataLoader(test_dataset, 1, num_workers=1, shuffle=False, collate_fn=collate_fn)
print('Loading annotation for test...')
annos, imgs = load_path(args.data_root, "refined_images", args.test_anno_folders)
images = load_images(imgs)
annotations_path = load_annotations_path(annos, images)
anno = load_annotations(annotations_path)
imagenames, recs = get_anno_recs(anno, each_flag=True)

Loading dataset for test ...
Loading annotation for test...


### Get model config

In [8]:
cfg = insect_refinedet[str(args.input_size)]

### Load model

In [9]:
print("Loading model for test ...")
model = build_refinedet('test', insect_refinedet, args.input_size, args.tcb_layer_num, num_classes=13).cuda()
load_name = pj(args.model_root, 'RefineDet{}_{}.pth'.format(args.input_size, "final"))
model.load_state_dict(torch.load(load_name))

Loading model for test ...


In [10]:
model

RefineDet(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (17): Conv2d

# --- result analysis ---

- classification using detecter

In [11]:
result = test_prediction_with_cls(model, test_data_loader, args.crop_num, 13, nms_thresh=0.3)
result = refine_result_by_ovthresh_with_cls(result, 13, ovthresh=0.3)
cls_lbl_dic = {1:6, 2:0, 3:1, 4:2, 5:6, 6:3, 7:4, 8:5, 9:6, 10:6, 11:6, 12:6}
result = result_formatter_with_cls(result, cls_lbl_dic)

In [12]:
gt_dict, npos = create_imwise_gt(recs)

In [13]:
# arrange index to label
# not considered label = -1
name2lbl = {
    'Coleoptera': 6, 
    'Diptera': 0, 
    'Ephemeridae': 1, 
    'Ephemeroptera': 2, 
    'Hemiptera': 6, 
    'Lepidoptera': 3, 
    'Plecoptera': 4, 
    'Trichoptera': 5, 
    'medium insect': 6, 
    'small insect': 6, 
    'snail': 6, 
    'spider': 6
}

In [14]:
accs, recalls, precisions = get_cls_accuracy_per_class_with_cls(result, gt_dict, name2lbl, add_divide_model=False, n_class_when_not_use_divide_model=7)

In [15]:
np.mean(accs)

0.5571356124096062

In [16]:
np.mean(recalls)

0.6795592400431111

In [17]:
np.mean(precisions)

0.5773701833202398

In [18]:
accs

[0.5428571428571428,
 0.75,
 0.4583333333333333,
 0.35294117647058826,
 0.5,
 0.6153846153846154,
 0.6111111111111112,
 0.6923076923076923,
 0.3684210526315789,
 0.68]

In [19]:
recalls

array([0.72727273, 0.38461538, 0.61290323, 0.75      , 0.83333333,
       0.76923077])

In [20]:
precisions

array([0.59701493, 0.83333333, 0.40425532, 0.65217391, 0.26315789,
       0.71428571])